<a href="https://colab.research.google.com/github/fastdatascience/clinical_trial_risk/blob/fixes_nov_2022/train/ctgov/TrainNumArmsClassifier_21_Spacy_Textcat_prefiltered_fewer_bins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_annotations = pd.read_csv("/content/drive/MyDrive/data/filtered_for_phase_arms_subjects.csv.bz2")

# Get data

In [ ]:
df_annotations = df_annotations[~df_annotations.num_arms.isna()]

In [ ]:
df_annotations["num_arms"].value_counts()

2.0     5228
1.0     3086
3.0     1456
4.0      908
5.0      315
6.0      251
7.0       83
8.0       80
9.0       58
10.0      31
12.0      27
11.0      17
13.0      10
14.0       8
15.0       7
18.0       5
16.0       4
20.0       3
17.0       3
23.0       2
21.0       2
29.0       1
19.0       1
34.0       1
38.0       1
30.0       1
40.0       1
22.0       1
Name: num_arms, dtype: int64

In [ ]:
import numpy as np
for i in np.quantile(df_annotations["num_arms"], [0, 0.2, 0.4, 0.6, 0.8, 1]):
  print (int(np.float(i)))

1
1
2
2
3
40


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
def get_num_arms_clean(num):
    if pd.isna(num):
        return None
    if num >= 3:
        return "3+"
    return str(int(num))
df_annotations["num_arms_clean"] = df_annotations["num_arms"].apply(get_num_arms_clean)

In [ ]:
df_annotations["num_arms_clean"].value_counts()

2     5228
3+    3277
1     3086
Name: num_arms_clean, dtype: int64

In [ ]:
labels = list(sorted(set(df_annotations["num_arms_clean"]), key =lambda x : int(re.sub(r'\D.*', '', x))))
labels

['1', '2', '3+']

In [ ]:
import spacy
nlp = spacy.blank("en")
from spacy.tokens import DocBin

docs_train = []
for i in range(10):
  docs_train.append(DocBin())
docs_val = DocBin()

for idx in range(len(df_annotations)):
  if idx % 100 == 0:
    print (idx, len(df_annotations))
  if df_annotations.train_val.iloc[idx] == "train":
    docs_list = docs_train[idx % 10]
  else:
    docs_list = docs_val
  text = str(df_annotations.text.iloc[idx])
  if len(text) > 100000:
      text = text[:100000]
  doc = nlp(text)

  cats = {}
  for a in labels:
      cats[a] = 0
  if pd.isna(df_annotations.num_arms_clean.iloc[idx]):
    continue
  cats[df_annotations.num_arms_clean.iloc[idx]] = 1
  doc.cats = cats

  docs_list.add(doc)

0 11591
100 11591
200 11591
300 11591
400 11591
500 11591
600 11591
700 11591
800 11591
900 11591
1000 11591
1100 11591
1200 11591
1300 11591
1400 11591
1500 11591
1600 11591
1700 11591
1800 11591
1900 11591
2000 11591
2100 11591
2200 11591
2300 11591
2400 11591
2500 11591
2600 11591
2700 11591
2800 11591
2900 11591
3000 11591
3100 11591
3200 11591
3300 11591
3400 11591
3500 11591
3600 11591
3700 11591
3800 11591
3900 11591
4000 11591
4100 11591
4200 11591
4300 11591
4400 11591
4500 11591
4600 11591
4700 11591
4800 11591
4900 11591
5000 11591
5100 11591
5200 11591
5300 11591
5400 11591
5500 11591
5600 11591
5700 11591
5800 11591
5900 11591
6000 11591
6100 11591
6200 11591
6300 11591
6400 11591
6500 11591
6600 11591
6700 11591
6800 11591
6900 11591
7000 11591
7100 11591
7200 11591
7300 11591
7400 11591
7500 11591
7600 11591
7700 11591
7800 11591
7900 11591
8000 11591
8100 11591
8200 11591
8300 11591
8400 11591
8500 11591
8600 11591
8700 11591
8800 11591
8900 11591
9000 11591
9100 11591


In [ ]:
!mkdir train.spacy
!mkdir /content/drive/MyDrive/data/arms_21_train.spacy


mkdir: cannot create directory ‘/content/drive/MyDrive/data/arms_21_train.spacy’: File exists


In [ ]:
for idx, doc_bin_train in enumerate(docs_train):
  doc_bin_train.to_disk(f"train.spacy/train_doc_bin{idx}.spacy")
  doc_bin_train.to_disk(f"/content/drive/MyDrive/data/arms_21_train.spacy/train_doc_bin{idx}.spacy")

In [ ]:
!ls /content/drive/MyDrive/data/arms_21_train.spacy

train_doc_bin0.spacy  train_doc_bin4.spacy  train_doc_bin8.spacy
train_doc_bin1.spacy  train_doc_bin5.spacy  train_doc_bin9.spacy
train_doc_bin2.spacy  train_doc_bin6.spacy
train_doc_bin3.spacy  train_doc_bin7.spacy


In [ ]:
docs_val.to_disk("dev.spacy")

In [ ]:
docs_val.to_disk("/content/drive/MyDrive/data/arms_21_dev.spacy")

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
%%javascript
Jupyter.notebook.session.delete();

In [ ]:
!python -m spacy train /content/drive/MyDrive/data/spacy_textcat_is_international_11.cfg --output /content/drive/MyDrive/data/output_arms_21 --paths.train /content/drive/MyDrive/data/arms_21_train.spacy --paths.dev  /content/drive/MyDrive/data/arms_21_dev.spacy --gpu-id 0

In [ ]:
import spacy
nlp = spacy.load("/content/drive/MyDrive/data/output_arms_21/model-best")

In [ ]:
df_val = df_annotations[df_annotations.train_val=="val"]

predictions = []
for idx in range(len(df_val)):
    doc = nlp(df_val.text.apply(str).iloc[idx])
    predictions.append(doc.cats)

In [ ]:
# pred_proba = []
# for idx in range(len(df_val)):
#     pred_proba.append([predictions[idx][a] for a in range(num_classes)])

In [ ]:
predictions = []
for idx in range(len(df_val)):
    doc = nlp(df_val.text.apply(str).iloc[idx])
    predictions.append(doc.cats)

In [ ]:
df_val["y_pred"] = [max(p, key=p.get) for p in predictions]

In [ ]:
num_arms_ground_truths = df_val.num_arms_clean
pred_num_arms = df_val["y_pred"]

In [ ]:
acc = accuracy_score(num_arms_ground_truths, pred_num_arms)
print (f"Num arms accuracy {acc}")

In [ ]:
ConfusionMatrixDisplay.from_predictions(num_arms_ground_truths, pred_num_arms, labels=labels)
plt.xticks(rotation=90)
;

In [ ]:
all_aucs = []
from sklearn.metrics import roc_auc_score
for i in labels:
  roc_auc = roc_auc_score(num_arms_ground_truths == i, [p[i] for p in predictions])
  print (f"Num arms ROC AUC {roc_auc} for {i} arms")
  all_aucs.append(roc_auc)
print ("Average", np.mean(all_aucs))

In [ ]:
plt.figure()
lw = 2
    
for i in labels:
    fpr, tpr, _ = roc_curve(num_arms_ground_truths == i, [p[i] for p in predictions])
    roc_auc = auc(fpr, tpr)
    plt.plot(
        fpr,
        tpr,
        # color="darkorange",
        lw=lw,
        label=f"ROC curve {i} arms (area = {roc_auc:0.2f})"
    )
    plt.plot([0, 1], [0, 1], color="gray", alpha=0.5, lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()